In [ ]:
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import normalize
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras import layers
from keras.layers import GaussianNoise
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from google.colab import drive

# Load Data

In [ ]:
# Loading the data
(training_images, training_labels), (testing_images, testing_labels) = mnist.load_data()

# checking if the variables are exactly equal to what is expected
assert training_images.shape == (60000, 28, 28)
assert testing_images.shape == (10000, 28, 28)
assert training_labels.shape == (60000,)
assert testing_labels.shape == (10000,)

# add the first 4 images to a plot
for i in range(4):
    plt.subplot(1, 4, (i+1), frameon=False)
    plt.title(f'label={training_labels[i]}')
    plt.imshow(training_images[i], cmap = plt.get_cmap('binary'))

plt.show()

# print image 1
print('Image 1 array')
print(training_images[0])

# flatten 3D images into 1D vectors
training_images = training_images.reshape((training_images.shape[0], 28, 28, 1))
testing_images = testing_images.reshape((testing_images.shape[0], 28, 28, 1))

# Convert vector to binary class
training_labels_categorical = np_utils.to_categorical(training_labels)
testing_labels_categorical = np_utils.to_categorical(testing_labels)

# normalize values to scale them between 0-1, makes it easier for model
# values are in greyscale so will be between 0 and 255
training_images = training_images / 255
testing_images = testing_images / 255

# add additional images to the training set
# def augment_data():
#   aug = tf.keras.preprocessing.image.ImageDataGenerator(
#   rotation_range=180,
#   zoom_range=0.05,
#   width_shift_range=0.1,
#   height_shift_range=0.1,
#   shear_range=0.15,
#   horizontal_flip=False,
#   fill_mode="nearest")
#   return aug

# Create Model

In [ ]:
# create the base model
model = Sequential()
model.add(tf.keras.Input(shape=(28, 28, 1)))
# add noise to the images
model.add(GaussianNoise(0.1))
# extracts certain features of the image, eg diagonal or horizontal lines
model.add(Conv2D(32, kernel_size=(3, 3) , activation = 'relu', padding='same'))
# Reduces dimensions of the image by taking the max value of each area eg 2x2
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Dropout(0.5))
# second Conv2D layer which will extract more complex features
model.add(Conv2D(64, kernel_size=(3, 3) , activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Dropout(0.5))
# third Conv2D layer which will extract more complex features
model.add(Conv2D(128, kernel_size=(3, 3) , activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Dropout(0.5))
# convert image into 1D binary class
model.add(Flatten())

# add fully connected layer. The purpose of this is to add some selection
# inteligence to the convulted image
model.add(Dense(128, activation='relu'))

# converts output into a value that can be used as a probability
model.add(Dense(10, activation='softmax'))

# TODO : visualise the model

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
for i in range(len(model.layers)):
    layer = model.layers[i]
    if 'conv' in layer.name:
      print(i , layer.name , layer.output.shape)

1 conv2d_3 (None, 28, 28, 32)
4 conv2d_4 (None, 14, 14, 64)
7 conv2d_5 (None, 7, 7, 128)


In [ ]:
drive.mount('/gdrive')
save_weights_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/gdrive/MyDrive/AI/weights',
    save_weights_only=True,
    verbose=1,
    save_freq=3*1875)

# Train Model

In [ ]:

training_history = model.fit(training_images, 
                             training_labels_categorical, 
                             epochs=3, 
                             validation_data=(testing_images, 
                                              testing_labels_categorical))
                            #  callbacks=[save_weights_callback])

plt.plot(training_history.history['accuracy'])
plt.plot(training_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Visualise Trained Model

In [ ]:
# get filter weights of the first convolutional layer
filters, biases = model.layers[1].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# 1st layer has 32 filters (figsize is in inches) https://www.pythonpool.com/matplotlib-figsize/
# TODO: Find out how to add title to the image
plt.figure(figsize=(20,2))
for i in range(32):
  ax = plt.subplot(2, 16, i+1)
  f = filters[:, :, :, i-1]
  plt.imshow(f[:, :, 0], cmap='binary')
  ax.set_xticks([])
  ax.set_yticks([])

print('1st convolution layer')
plt.show()

# get filter weights of the first convolutional layer
filters, biases = model.layers[4].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# display the second convolutional layer
plt.figure(figsize=(20,4))
for i in range(64):
  ax = plt.subplot(4, 16, i+1)
  f = filters[:, :, :, i-1]
  plt.imshow(f[:, :, 0], cmap='binary')
  ax.set_xticks([])
  ax.set_yticks([])

print('2nd convolution layer')
plt.show()

# Use model to predict

In [ ]:
prediction_count = 20
prediction_list = model.predict(testing_images[0:prediction_count])
x_img = testing_images.reshape((10000, 28, 28))

for i in range(prediction_count):
  plt.subplot(3, 1, 1, frameon=False)
  plt.title(f'label={testing_labels[i]}, predict={np.argmax(prediction_list[i])}')
  plt.imshow(x_img[i], cmap = plt.get_cmap('binary'))
  plt.show()  

#for sublist in prediction_list:
#  list_pos = np.where(prediction_list == sublist)[0][0]
#  print(f"Prediction: {np.argmax(sublist)} Actual: {np.argmax(y_data[list_pos])}")


# Save Model

In [ ]:
drive.mount('/gdrive')
reconstructed_model = model.save('/gdrive/MyDrive/model')